## WCS solutions for AuxTel - 29-May-24

Craig Lage 

In [ ]:
import sys, time, os, asyncio, glob
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as pf
from astropy.time import Time, TimeDelta
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5, SkyCoord
import astropy.units as u
from lsst.obs.lsst.translators.latiss import AUXTEL_LOCATION
from lsst.obs.lsst.translators.lsst import SIMONYI_LOCATION
from astroquery.astrometry_net import AstrometryNet
from lsst.summit.utils.astrometry.anet import CommandLineSolver
import lsst.summit.utils.butlerUtils as butlerUtils
from lsst.summit.utils.bestEffort import BestEffortIsr
from lsst.summit.utils.plotting import plot
from lsst.summit.utils import quickSmooth
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm
from lsst.summit.utils.astrometry.utils import runCharactierizeImage, filterSourceCatOnBrightest
import lsst.geom as geom

In [ ]:
butler = butlerUtils.makeDefaultLatissButler(embargo=False)
bestEffort = BestEffortIsr(embargo=False)
solver = CommandLineSolver()
# So we'll use the remote astrometry.net solver
ast = AstrometryNet()
ast.api_key = 'xxawwhvleirxcswx'

In [ ]:
dataId = dict(day_obs=20240528, detector=0, seq_num=323)
bestEffort.getExposure(dataId)
dataType = 'quickLookExp'
exp = butler.get(dataType, dataId)
mData = butler.get('raw.metadata', dataId=dataId)

In [ ]:
%matplotlib inline
test = plot(exp)

In [ ]:
results = runCharactierizeImage(exp, 5, 10)
sourceCat = results.sourceCat
filteredCat = filterSourceCatOnBrightest(results.sourceCat, 0.2, maxSources=100)
x = filteredCat.getColumnView().getX()
y = filteredCat.getColumnView().getY()
wcs = exp.getWcs()
image_width = int(mData['DETSIZE'].strip('[]').split(',')[0].split(':')[1])
image_height = int(mData['DETSIZE'].strip('[]').split(',')[1].split(':')[1])
scale_units = 'arcsecperpix'
scale_type='ev' # ev means submit estimate and % error
scale_est = wcs.getPixelScale().asArcseconds()
scale_err = 2.0
center_ra = mData['RA']
center_dec = mData['DEC']
radius = 0.5
print(len(filteredCat))

In [ ]:
wcs_anet = ast.solve_from_source_list(x, y, 
                                        image_width, image_height, scale_units=scale_units,
                                        scale_type=scale_type, scale_est=scale_est, scale_err=scale_err,
                                        center_ra=center_ra, center_dec=center_dec, radius=radius,
                                        crpix_center=True, solve_timeout=240)
print(wcs_anet['CRVAL1'])

In [ ]:
# Get the DM wcs coordinates of the center pixel:
print(dataId)
DM_center = wcs.pixelToSky(wcs_anet['CRPIX1'], wcs_anet['CRPIX2'])
DM_RA_Dec = (DM_center.getRa().asDegrees(), DM_center.getDec().asDegrees())
print(f"Header center pix RA/Dec = {DM_RA_Dec[0]}, {DM_RA_Dec[1]} degrees")
Anet_RA_Dec = (wcs_anet['CRVAL1'], wcs_anet['CRVAL2'])
print(f"Astrometry.net center pix RA/Dec = {Anet_RA_Dec[0]}, {Anet_RA_Dec[1]} degrees")
Diff = ((DM_RA_Dec[0] - Anet_RA_Dec[0]) * 3600.0, (DM_RA_Dec[1] - Anet_RA_Dec[1]) * 3600.0)
print(f"Difference center pix RA/Dec = {Diff[0]}, {Diff[1]} arcseconds")

In [ ]:
time = Time((Time(mData["DATE-BEG"], scale='tai').unix_tai + Time(mData["DATE-END"], scale='tai').unix_tai)/2.0, format='unix_tai', scale='tai')
altAz = AltAz(obstime=time, location=AUXTEL_LOCATION)
DMSkyLocation = SkyCoord(DM_center.getRa().asDegrees() * u.deg, DM_center.getDec().asDegrees() * u.deg)
DMObsAltAz = DMSkyLocation.transform_to(altAz)
DMaz = DMObsAltAz.az.deg
DMaz = DMaz % 360
DMel = DMObsAltAz.alt.deg
AnetSkyLocation = SkyCoord(Anet_RA_Dec[0] * u.deg, Anet_RA_Dec[1] * u.deg)
AnetObsAltAz = AnetSkyLocation.transform_to(altAz)
Anetaz = AnetObsAltAz.az.deg
Anetaz = Anetaz % 360
Anetel = AnetObsAltAz.alt.deg
#print(DMaz, DMel)
#print(Anetaz, Anetel)
AzDiff = DMaz - Anetaz
if AzDiff > 360.:
    AzDiff -= 360.0
Diff = (AzDiff * 3600.0, (DMel - Anetel) * 3600.0)
print(dataId)
print(f"Az = {DMaz}, El = {DMel}")
print(f"Difference center pix Az/El = {Diff[0]}, {Diff[1]} arcseconds")

In [ ]:
dataId = dict(day_obs=20240419, detector=0, seq_num=477)
bestEffort.getExposure(dataId)
dataType = 'quickLookExp'
exp = butler.get(dataType, dataId)
mData = butler.get('raw.metadata', dataId=dataId)

In [ ]:
%matplotlib inline
test = plot(exp)

In [ ]:
results = runCharactierizeImage(exp, 5, 10)
sourceCat = results.sourceCat
filteredCat = filterSourceCatOnBrightest(results.sourceCat, 0.2, maxSources=100)
x = filteredCat.getColumnView().getX()
y = filteredCat.getColumnView().getY()
wcs = exp.getWcs()
image_width = int(mData['DETSIZE'].strip('[]').split(',')[0].split(':')[1])
image_height = int(mData['DETSIZE'].strip('[]').split(',')[1].split(':')[1])
scale_units = 'arcsecperpix'
scale_type='ev' # ev means submit estimate and % error
scale_est = wcs.getPixelScale().asArcseconds()
scale_err = 2.0
center_ra = mData['RA']
center_dec = mData['DEC']
radius = 0.5
print(len(filteredCat))

In [ ]:
wcs_anet = ast.solve_from_source_list(x, y, 
                                        image_width, image_height, scale_units=scale_units,
                                        scale_type=scale_type, scale_est=scale_est, scale_err=scale_err,
                                        center_ra=center_ra, center_dec=center_dec, radius=radius,
                                        crpix_center=True, solve_timeout=240)
print(wcs_anet['CRVAL1'])

In [ ]:
wcs_anet

In [ ]:
# Get the DM wcs coordinates of the center pixel:
print(dataId)
DM_center = wcs.pixelToSky(wcs_anet['CRPIX1'], wcs_anet['CRPIX2'])
DM_RA_Dec = (DM_center.getRa().asDegrees(), DM_center.getDec().asDegrees())
print(f"Header center pix RA/Dec = {DM_RA_Dec[0]}, {DM_RA_Dec[1]} degrees")
Anet_RA_Dec = (wcs_anet['CRVAL1'], wcs_anet['CRVAL2'])
print(f"Astrometry.net center pix RA/Dec = {Anet_RA_Dec[0]}, {Anet_RA_Dec[1]} degrees")
Diff = ((DM_RA_Dec[0] - Anet_RA_Dec[0]) * 3600.0, (DM_RA_Dec[1] - Anet_RA_Dec[1]) * 3600.0)
print(f"Difference center pix RA/Dec = {Diff[0]}, {Diff[1]} arcseconds")
DMSkyLocation = SkyCoord(DM_center.getRa().asDegrees() * u.deg, DM_center.getDec().asDegrees() * u.deg)
DMObsAltAz = DMSkyLocation.transform_to(altAz)
DMaz = DMObsAltAz.az.deg
DMaz = DMaz%360
DMel = DMObsAltAz.alt.deg
AnetSkyLocation = SkyCoord(Anet_RA_Dec[0] * u.deg, Anet_RA_Dec[1] * u.deg)
AnetObsAltAz = AnetSkyLocation.transform_to(altAz)
Anetaz = AnetObsAltAz.az.deg
Anetaz = Anetaz%360 
Anetel = AnetObsAltAz.alt.deg
#print(DMaz, DMel)
#print(Anetaz, Anetel)
deltaAz = geom.Angle.separation(geom.Angle(DMaz, geom.degrees), geom.Angle(Anetaz, geom.degrees))
deltaEl = geom.Angle.separation(geom.Angle(DMel, geom.degrees), geom.Angle(Anetel, geom.degrees))
print(dataId)
print(f"Az = {DMaz}, El = {DMel}")
print(f"Difference center pix Az/El = {deltaAz.asArcseconds()}, {deltaEl.asArcseconds()} arcseconds")


In [ ]:
Azs = []
Els = []
dAzs = []
dEls = []

#for seqNum in [323, 360, 362, 364, 372]:
#    dataId = dict(day_obs=20240528, detector=0, seq_num=seqNum)
for seqNum in [477, 506, 578]:
    dataId = dict(day_obs=20240530, detector=0, seq_num=seqNum)
    bestEffort.getExposure(dataId)
    dataType = 'quickLookExp'
    exp = butler.get(dataType, dataId)
    mData = butler.get('raw.metadata', dataId=dataId)
    results = runCharactierizeImage(exp, 5, 10)
    sourceCat = results.sourceCat
    filteredCat = filterSourceCatOnBrightest(results.sourceCat, 0.2, maxSources=100)
    x = filteredCat.getColumnView().getX()
    y = filteredCat.getColumnView().getY()
    wcs = exp.getWcs()
    image_width = int(mData['DETSIZE'].strip('[]').split(',')[0].split(':')[1])
    image_height = int(mData['DETSIZE'].strip('[]').split(',')[1].split(':')[1])
    scale_units = 'arcsecperpix'
    scale_type='ev' # ev means submit estimate and % error
    scale_est = wcs.getPixelScale().asArcseconds()
    scale_err = 2.0
    center_ra = mData['RA']
    center_dec = mData['DEC']
    radius = 0.5
    print(seqNum, len(filteredCat))
    wcs_anet = ast.solve_from_source_list(x, y, 
                                            image_width, image_height, scale_units=scale_units,
                                            scale_type=scale_type, scale_est=scale_est, scale_err=scale_err,
                                            center_ra=center_ra, center_dec=center_dec, radius=radius,
                                            crpix_center=True, solve_timeout=240)
    print(seqNum, wcs_anet['CRVAL1'])
    time = Time((Time(mData["DATE-BEG"], scale='tai').unix_tai + Time(mData["DATE-END"], scale='tai').unix_tai)/2.0, format='unix_tai', scale='tai')
    altAz = AltAz(obstime=time, location=AUXTEL_LOCATION)
    DM_center = wcs.pixelToSky(wcs_anet['CRPIX1'], wcs_anet['CRPIX2'])
    DM_RA_Dec = (DM_center.getRa().asDegrees(), DM_center.getDec().asDegrees())
    print(f"Header center pix RA/Dec = {DM_RA_Dec[0]}, {DM_RA_Dec[1]} degrees")
    Anet_RA_Dec = (wcs_anet['CRVAL1'], wcs_anet['CRVAL2'])
    print(f"Astrometry.net center pix RA/Dec = {Anet_RA_Dec[0]}, {Anet_RA_Dec[1]} degrees")
    Diff = ((DM_RA_Dec[0] - Anet_RA_Dec[0]) * 3600.0, (DM_RA_Dec[1] - Anet_RA_Dec[1]) * 3600.0)
    print(f"Difference center pix RA/Dec = {Diff[0]}, {Diff[1]} arcseconds")
    DMSkyLocation = SkyCoord(DM_center.getRa().asDegrees() * u.deg, DM_center.getDec().asDegrees() * u.deg)
    DMObsAltAz = DMSkyLocation.transform_to(altAz)
    DMaz = DMObsAltAz.az.deg
    DMaz = DMaz%360
    DMel = DMObsAltAz.alt.deg
    AnetSkyLocation = SkyCoord(Anet_RA_Dec[0] * u.deg, Anet_RA_Dec[1] * u.deg)
    AnetObsAltAz = AnetSkyLocation.transform_to(altAz)
    Anetaz = AnetObsAltAz.az.deg
    Anetaz = Anetaz%360 
    Anetel = AnetObsAltAz.alt.deg
    #print(DMaz, DMel)
    #print(Anetaz, Anetel)
    deltaAz = geom.Angle.separation(geom.Angle(DMaz, geom.degrees), geom.Angle(Anetaz, geom.degrees))
    deltaEl = geom.Angle.separation(geom.Angle(DMel, geom.degrees), geom.Angle(Anetel, geom.degrees))
    print(dataId)
    print(f"Az = {DMaz}, El = {DMel}")
    print(f"Difference center pix Az/El = {deltaAz.asArcseconds()}, {deltaEl.asArcseconds()} arcseconds")
    Azs.append(DMaz)
    Els.append(DMel)
    dAzs.append(deltaAz.asArcseconds())
    dEls.append(deltaEl.asArcseconds())

In [ ]:
fig, axs = plt.subplots(2,2,figsize=(10,10))
plt.suptitle(f"AuxTel Pointing errors from Astrometry.net, {dataId['day_obs']}", fontsize=18)
axs[0][0].scatter(Azs, dAzs)
axs[0][0].set_xlabel("Azimuth(degrees)")
axs[0][0].set_ylabel("DeltaAz (arcseconds)")
axs[0][0].set_ylim(-200, 1500)
axs[0][1].scatter(Azs, dEls)
axs[0][1].set_xlabel("Azimuth(degrees)")
axs[0][1].set_ylabel("DeltaAEl (arcseconds)")
axs[0][1].set_ylim(-200, 1500)
axs[1][0].scatter(Els, dAzs)
axs[1][0].set_xlabel("Elevation(degrees)")
axs[1][0].set_ylabel("DeltaAz (arcseconds)")
axs[1][0].set_ylim(-200, 1500)
axs[1][1].scatter(Els, dEls)
axs[1][1].set_xlabel("Elevation(degrees)")
axs[1][1].set_ylabel("DeltaAEl (arcseconds)")
axs[1][1].set_ylim(-200, 1500)
plt.savefig(f"/home/cslage/DATA/AuxTel_Pointing_Errors_{dataId['day_obs']}.png")

In [ ]:
from scipy.optimize import minimize

In [ ]:
def func(params, args):
    # Determines best shift to match up applied and measured forces
    [Azs, Els, ds] = args
    [a1, a2, a3] = params
    diff = 0.0
    for i in range(len(Azs)):
        model = a1 * Azs[i] + a2 * Els[i] + a3
        diff += (model - ds[i])**2
    return diff


In [ ]:
args = [Azs, Els, dAzs]
param0 = [5.0, 5.0, 100.0]
        
bestFit = minimize(func, param0, args=args, method='Powell')
bestFit.x

In [ ]:
model = bestFit.x[0] * np.array(Azs) +  bestFit.x[1] * np.array(Els) +  bestFit.x[2]

In [ ]:
%matplotlib inline
plt.scatter(model, dAzs)


In [ ]:
from mpl_toolkits.mplot3d import Axes3D

x = np.linspace(0, 360.0, 100)
y = np.linspace(50, 80, 100)
# create x,y
xx, yy = np.meshgrid(x,y)

# calculate corresponding z
z = bestFit.x[0] * xx +  bestFit.x[1] * yy +  bestFit.x[2]

fig = plt.figure() 
ax = fig.add_subplot(111, projection='3d') 
ax.view_init(elev=10., azim=90)
ax.scatter(Azs, Els, dEls,  color='green', marker = 'o', s=200) 
ax.plot_surface(xx, yy, z, color='red', alpha=0.5) 
ax.set_xlabel('Azimuth') 
ax.set_ylabel('Elevation') 
ax.set_zlabel('Azimuth Error') 
plt.show()


